## Projeto Aluguel de Veículos Elétricos

In [41]:
# Pacotes necessários
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import time
import datetime as dt
from pyspark.sql.functions import concat_ws, to_timestamp
from pyspark.sql.functions import when, expr
from pyspark.sql.functions import col, round
from pyspark.sql.functions import avg


In [2]:
# Sessão Spark
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
# Cria o dataframe
arquivo = "Projeto_Veiculos/dataset.csv"
spark_df = spark.read.option("header", True).csv(arquivo)

In [6]:
type(spark_df)

pyspark.sql.dataframe.DataFrame

In [13]:
spark_df.count()

686299

In [14]:
len(spark_df.columns)

9

In [15]:
spark_df.columns

['Genero_Usuario',
 'Idade_Usuario',
 'Veiculo',
 'Loja_Aluguel',
 'Data_Aluguel',
 'Hora_Aluguel',
 'Loja_Devolucao',
 'Data_Devolucao',
 'Hora_Devolucao']

In [11]:
spark_df.show(5)

+--------------+-------------+-------+------------+------------+------------+--------------+--------------+--------------+
|Genero_Usuario|Idade_Usuario|Veiculo|Loja_Aluguel|Data_Aluguel|Hora_Aluguel|Loja_Devolucao|Data_Devolucao|Hora_Devolucao|
+--------------+-------------+-------+------------+------------+------------+--------------+--------------+--------------+
|             M|           44|   4357|         442|  01/02/2020|     0:00:38|           116|    01/02/2020|       0:35:17|
|             M|           22|  12083|          66|  01/02/2020|     0:00:53|            37|    01/02/2020|       0:06:23|
|             M|           29|  11562|         331|  01/02/2020|     0:00:55|           341|    01/02/2020|       0:26:47|
|             M|           27|  10206|         164|  01/02/2020|     0:01:18|            35|    01/02/2020|       0:16:51|
|             M|           27|  10101|         120|  01/02/2020|     0:01:18|            47|    01/02/2020|       0:12:39|
+--------------+

In [28]:
spark_df.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Idade_Usuario: string (nullable = true)
 |-- Veiculo: string (nullable = true)
 |-- Loja_Aluguel: string (nullable = true)
 |-- Loja_Devolucao: string (nullable = true)
 |-- datahora_aluguel: timestamp (nullable = true)
 |-- datahora_devolucao: timestamp (nullable = true)



In [16]:
spark_df.describe().show()

+-------+--------------+-----------------+------------------+-----------------+------------+------------+------------------+--------------+--------------+
|summary|Genero_Usuario|    Idade_Usuario|           Veiculo|     Loja_Aluguel|Data_Aluguel|Hora_Aluguel|    Loja_Devolucao|Data_Devolucao|Hora_Devolucao|
+-------+--------------+-----------------+------------------+-----------------+------------+------------+------------------+--------------+--------------+
|  count|        686299|           686299|            686299|           686299|      686299|      686299|            686299|        686299|        686299|
|   mean|          NULL|35.28143709957322|  9376.32784981473|184.6764908589405|        NULL|        NULL|183.73291961666854|          NULL|          NULL|
| stddev|          NULL|10.39871787138301|2524.5212819960666|134.8961691532454|        NULL|        NULL|  134.574154763723|          NULL|          NULL|
|    min|             F|              119|             10000|         

### Limpeza e Transformação

In [18]:
# Cria colunas de data/hora concatenadas
spark_df = spark_df.withColumn(
    'datahora_aluguel',
    to_timestamp(concat_ws(' ', 'Data_Aluguel', 'Hora_Aluguel'), 'dd/MM/yyyy HH:mm:ss')
)

spark_df = spark_df.withColumn(
    'datahora_devolucao',
    to_timestamp(concat_ws(' ', 'Data_Devolucao', 'Hora_Devolucao'), 'dd/MM/yyyy HH:mm:ss')
)

In [24]:
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

In [27]:
spark_df.show(5)

+--------------+-------------+-------+------------+--------------+-------------------+-------------------+
|Genero_Usuario|Idade_Usuario|Veiculo|Loja_Aluguel|Loja_Devolucao|   datahora_aluguel| datahora_devolucao|
+--------------+-------------+-------+------------+--------------+-------------------+-------------------+
|             M|           44|   4357|         442|           116|2020-02-01 00:00:38|2020-02-01 00:35:17|
|             M|           22|  12083|          66|            37|2020-02-01 00:00:53|2020-02-01 00:06:23|
|             M|           29|  11562|         331|           341|2020-02-01 00:00:55|2020-02-01 00:26:47|
|             M|           27|  10206|         164|            35|2020-02-01 00:01:18|2020-02-01 00:16:51|
|             M|           27|  10101|         120|            47|2020-02-01 00:01:18|2020-02-01 00:12:39|
+--------------+-------------+-------+------------+--------------+-------------------+-------------------+
only showing top 5 rows



In [26]:
# Remove colunas antigas
spark_df = spark_df.drop('Data_Aluguel', 'Hora_Aluguel', 'Data_Devolucao', 'Hora_Devolucao')

In [31]:
# Cria faixas etárias
spark_df = spark_df.withColumn(
    'Faixa_Etaria',
    when(spark_df.Idade_Usuario < 18, 'Menor de idade')
    .when(spark_df.Idade_Usuario < 30, 'Jovem adulto')
    .when(spark_df.Idade_Usuario < 45, 'Adulto')
    .when(spark_df.Idade_Usuario < 60, 'Meia-idade')
    .otherwise('Idoso')
)

In [32]:
spark_df.select('Idade_Usuario', 'Faixa_Etaria').distinct().orderBy('Idade_Usuario').show()

+-------------+--------------+
|Idade_Usuario|  Faixa_Etaria|
+-------------+--------------+
|          119|         Idoso|
|           16|Menor de idade|
|           17|Menor de idade|
|           18|  Jovem adulto|
|           19|  Jovem adulto|
|           20|  Jovem adulto|
|           21|  Jovem adulto|
|           22|  Jovem adulto|
|           23|  Jovem adulto|
|           24|  Jovem adulto|
|           25|  Jovem adulto|
|           26|  Jovem adulto|
|           27|  Jovem adulto|
|           28|  Jovem adulto|
|           29|  Jovem adulto|
|           30|        Adulto|
|           31|        Adulto|
|           32|        Adulto|
|           33|        Adulto|
|           34|        Adulto|
+-------------+--------------+
only showing top 20 rows



### Análises

In [34]:
# Calcula a duração do aluguel
spark_df = spark_df.withColumn(
    'Duracao_Horas',
    round((col('datahora_devolucao').cast('long') - col('datahora_aluguel').cast('long')) / 3600, 2)
)

In [37]:
spark_df.select('datahora_aluguel', 'datahora_devolucao', 'Duracao_Horas').show(5)

+-------------------+-------------------+-------------+
|   datahora_aluguel| datahora_devolucao|Duracao_Horas|
+-------------------+-------------------+-------------+
|2020-02-01 00:00:38|2020-02-01 00:35:17|         0.58|
|2020-02-01 00:00:53|2020-02-01 00:06:23|         0.09|
|2020-02-01 00:00:55|2020-02-01 00:26:47|         0.43|
|2020-02-01 00:01:18|2020-02-01 00:16:51|         0.26|
|2020-02-01 00:01:18|2020-02-01 00:12:39|         0.19|
+-------------------+-------------------+-------------+
only showing top 5 rows



In [38]:
# Cria coluna Duracao_Minutos (parte inteira)
spark_df = spark_df.withColumn(
    'Duracao_Minutos', expr('int(Duracao_Horas * 60)')
)

In [39]:
spark_df.select('Duracao_Horas', 'Duracao_Minutos').show(5)

+-------------+---------------+
|Duracao_Horas|Duracao_Minutos|
+-------------+---------------+
|         0.58|             34|
|         0.09|              5|
|         0.43|             25|
|         0.26|             15|
|         0.19|             11|
+-------------+---------------+
only showing top 5 rows



In [44]:
# Média de duração do aluguel por faixa etária
spark_df.groupBy('Faixa_Etaria') \
    .agg(round(avg('Duracao_Minutos'), 2).alias('Media_Duracao_Minutos')) \
    .orderBy('Faixa_Etaria') \
    .show()

+--------------+---------------------+
|  Faixa_Etaria|Media_Duracao_Minutos|
+--------------+---------------------+
|        Adulto|                15.44|
|         Idoso|                14.13|
|  Jovem adulto|                15.44|
|    Meia-idade|                14.25|
|Menor de idade|                12.71|
+--------------+---------------------+



In [47]:
# Veículos mais alugados (Top 10)
spark_df.groupBy('Veiculo').count().orderBy('count', ascending=False).show(10)

+-------+-----+
|Veiculo|count|
+-------+-----+
|  10771|  217|
|  10810|  208|
|   7854|  193|
|   8463|  190|
|   4013|  189|
|  10907|  186|
|  10476|  185|
|  10442|  184|
|   7422|  184|
|   7892|  184|
+-------+-----+
only showing top 10 rows



In [46]:
# Lojas com mais aluguéis (Top 10)
spark_df.groupBy('Loja_Aluguel').count().orderBy('count', ascending=False).show(10)

+------------+-----+
|Loja_Aluguel|count|
+------------+-----+
|           1| 6309|
|          27| 6240|
|         271| 5265|
|          64| 4834|
|          41| 4633|
|         182| 4593|
|          18| 4384|
|          28| 4345|
|          36| 4202|
|          43| 4121|
+------------+-----+
only showing top 10 rows



In [48]:
# Tempo médio de aluguel por veículo (Top 10)
spark_df.groupBy('Veiculo').agg(round(avg('Duracao_Minutos'), 2).alias('Media_Duracao_Minutos')) \
    .orderBy('Media_Duracao_Minutos', ascending=False).show(10)

+-------+---------------------+
|Veiculo|Media_Duracao_Minutos|
+-------+---------------------+
|   9589|             120839.0|
|   9828|             108618.0|
|   7808|              1352.67|
|   7981|              1098.94|
|   7770|              1050.65|
|   7288|               233.67|
|   7586|               171.62|
|   9081|               151.01|
|   7117|               142.17|
|  12494|               141.89|
+-------+---------------------+
only showing top 10 rows



In [50]:
# Quantidade de aluguéis por faixa etária
spark_df.groupBy('Faixa_Etaria').count().orderBy('count', ascending=False).show()

+--------------+------+
|  Faixa_Etaria| count|
+--------------+------+
|        Adulto|327428|
|  Jovem adulto|233806|
|    Meia-idade|101529|
|         Idoso| 22204|
|Menor de idade|  1332|
+--------------+------+



## Fim